In [1]:
import requests
from bs4 import BeautifulSoup

website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url,'lxml')

Scraping the url is next

In [2]:
parsed_table_data = []
myTable = soup.find('table',{'class':'wikitable sortable'})
rows = myTable.findAll('tr')
for row in rows:
    children = row.findChildren(recursive=False)
    row_text = []
    for child in children:
        clean_text = child.text
        #This is to discard reference/citation links
        clean_text = clean_text.split('&#91;')[0]
        #This is to clean the header row of the sort icons
        clean_text = clean_text.split('&#160;')[-1]
        clean_text = clean_text.strip()
        row_text.append(clean_text)
    parsed_table_data.append(row_text)


After cleaning the data, let's take a look at the obtained list:

In [3]:
parsed_table_data[0:5]

[['Postcode', 'Borough', 'Neighbourhood'],
 ['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village']]

Now let's convert it to a dataframe. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [4]:
import pandas as pd
columns_names = ['PostalCode', 'Borough', 'Neighbourhood']
df = pd.DataFrame(parsed_table_data,columns = columns_names)
df = df.drop([0])
df.head()

,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


Let's only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [5]:
df = df[df.Borough != 'Not assigned']
df.head(10)

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 

In [6]:
 df.loc[df.Neighbourhood == 'Not assigned',['Borough']]['Borough']

9    Queen's Park
Name: Borough, dtype: object

In [7]:
df.loc[df.Neighbourhood == 'Not assigned',['Neighbourhood']] =  df.loc[df.Neighbourhood == 'Not assigned',['Borough']]['Borough']
df.head(10)

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Queen's Park
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


If more than one neighborhood can exist in one postal code area, these will be combined into one row with the neighborhoods separated with a comma.

In [8]:
df = df.groupby(['PostalCode','Borough']).agg(lambda x: ','.join(set(x))).reset_index()

In [9]:
df.head(10)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"West Hill,Morningside,Guildwood"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Ionview,Kennedy Park,East Birchmount Park"
7,M1L,Scarborough,"Golden Mile,Oakridge,Clairlea"
8,M1M,Scarborough,"Cliffside,Cliffcrest,Scarborough Village West"
9,M1N,Scarborough,"Cliffside West,Birch Cliff"


In [10]:
df.shape

(103, 3)

I COULD NOT EXPLORE THE LAT, LONG USING GEOCODER
OVER_QUERY_LIMIT was returned

In [11]:
#!conda install -c conda-forge geocoder --yes
#import geocoder # import geocoder
#latitude = []
#longitude = []
#for postal_code in df['PostalCode']:
#    # initialize your variable to None
#    lat_lng_coords = None
#    # loop until you get the coordinates
#    while(lat_lng_coords is None):
#      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#      lat_lng_coords = g.latlng
#    latitude = lat_lng_coords[0]
#    longitude = lat_lng_coords[1]

Let's explore the lat and long using the CSV file

In [12]:
!wget -q -O 'Geospatial_Coordinates.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [13]:
latlng_df = pd.read_csv('Geospatial_Coordinates.csv')
latlng_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Now let's rename the column to match the PostalCode Column in the original dataframe before we merge them

In [14]:
latlng_df.columns = ['PostalCode','Latitude','Longitude']
latlng_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In the following step, we will use pandas to merge the two tables based on the Postal Code column

In [15]:
neighborhoods = pd.merge(df,latlng_df[['PostalCode','Latitude','Longitude']],on='PostalCode')
neighborhoods.head(10)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"West Hill,Morningside,Guildwood",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Ionview,Kennedy Park,East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile,Oakridge,Clairlea",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside,Cliffcrest,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West,Birch Cliff",43.692657,-79.264848


In [16]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


Let's explore Toronto's neighbourhood and plot each neighbrouhood on the map

In [17]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'Toronto, ON'
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude


Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.17.0                     py_0    conda-forge


In [18]:
# create map of Toronto using Toronto's latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto


Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

In [19]:
import time
CLIENT_ID = '0KKTJKIPL1VUNXKP51HICZNDEDR5YU4BOCUE1J0UBOOQXJFC' # your Foursquare ID
CLIENT_SECRET = 'WWQXM1HSQL0KVAX3QXY5PUYTKYI2GC4TPYJ3OL1MKKS0HYC3' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        resultsCount = len(results)
        if resultsCount == 0:
            print("WARNING!!!!! Neighborhood returned ZERO results, will be removed")
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

toronto_venues = getNearbyVenues(names=neighborhoods['Neighbourhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )


Malvern,Rouge
Highland Creek,Rouge Hill,Port Union
West Hill,Morningside,Guildwood
Woburn
Cedarbrae
Scarborough Village
Ionview,Kennedy Park,East Birchmount Park
Golden Mile,Oakridge,Clairlea
Cliffside,Cliffcrest,Scarborough Village West
Cliffside West,Birch Cliff
Scarborough Town Centre,Dorset Park,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
L'Amoreaux East,Milliken,Steeles East,Agincourt North
Steeles West,L'Amoreaux West
Upper Rouge
WARNING!!!!! Neighborhood returned ZERO results, will be removed
Hillcrest Village
Henry Farm,Oriole,Fairview
Bayview Village
York Mills,Silver Hills
Newtonbrook,Willowdale
WARNING!!!!! Neighborhood returned ZERO results, will be removed
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Wilson Heights,Downsview North,Bathurst Manor
Northwood Park,York University
Downsview East,CFB Toronto
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Park

Let's check the size of the resulting dataframe

In [20]:
print(toronto_venues.shape)

(1329, 7)


Some of the neighborhoods did not return enough data from Foursquare, we will proceed by removing those neighbrohoods from the list.

In [21]:
toronto_venues_grouped = toronto_venues.groupby('Neighborhood').count()
toronto_venues_grouped.reset_index(inplace=True)
missing_set = list(set(neighborhoods['Neighbourhood']) - set(toronto_venues_grouped['Neighborhood']))
missing_df = neighborhoods.loc[neighborhoods['Neighbourhood'].isin(missing_set)]

missing_df

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
16,M1X,Scarborough,Upper Rouge,43.836125,-79.205636
21,M2M,North York,"Newtonbrook,Willowdale",43.789053,-79.408493
93,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242


In [22]:
for hood in list(missing_df['Neighbourhood']):
    indx = neighborhoods.index[neighborhoods['Neighbourhood'] == hood]
    neighborhoods.drop(indx,inplace=True)

In [23]:
neighborhoods.groupby('Neighbourhood').count().shape

(100, 4)

Let's check how many venues were returned for each neighborhood

In [24]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",30,30,30,30,30,30
Agincourt,4,4,4,4,4,4
"Bathurst Quay,Harbourfront West,Island airport,South Niagara,King and Spadina,CN Tower,Railway Lands",14,14,14,14,14,14
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",25,25,25,25,25,25
Berczy Park,30,30,30,30,30,30
Business reply mail Processing Centre969 Eastern,17,17,17,17,17,17
"Cabbagetown,St. James Town",30,30,30,30,30,30
Caledonia-Fairbanks,6,6,6,6,6,6


Let's find out how many unique categories can be curated from all the returned venues

In [25]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 239 uniques categories.


In [26]:
# Analyze Each Neighborhood

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns)
fixed_columns.remove("Neighborhood")
fixed_columns = ['Neighborhood'] + fixed_columns
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()



,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Organic Grocery,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Pool,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Malvern,Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Malvern,Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [27]:
#And let's examine the new dataframe size.
toronto_onehot.shape

(1329, 239)

In [28]:
#Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.shape

(100, 239)

In [29]:
num_top_venues = 5
for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Adelaide,King,Richmond----
                 venue  freq
0  American Restaurant  0.10
1           Steakhouse  0.10
2                Hotel  0.07
3     Asian Restaurant  0.07
4                 Café  0.07


----Agincourt----
               venue  freq
0             Lounge  0.25
1     Clothing Store  0.25
2     Breakfast Spot  0.25
3       Skating Rink  0.25
4  Accessories Store  0.00


----Bathurst Quay,Harbourfront West,Island airport,South Niagara,King and Spadina,CN Tower,Railway Lands----
              venue  freq
0   Airport Service  0.14
1  Airport Terminal  0.14
2    Airport Lounge  0.14
3   Harbor / Marina  0.07
4  Sculpture Garden  0.07


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Bank  0.25
2                 Café  0.25
3  Japanese Restaurant  0.25
4    Accessories Store  0.00


----Bedford Park,Lawrence Manor East----
                  venue  freq
0    Italian Restaurant  0.12
1  Fast Food Restaurant  0.08
2           C

                 venue  freq
0                Field  0.25
1         Hockey Arena  0.25
2              Dog Run  0.25
3                Trail  0.25
4  Monument / Landmark  0.00


----India Bazaar,The Beaches West----
                venue  freq
0      Sandwich Place  0.10
1                 Gym  0.05
2  Italian Restaurant  0.05
3         Pizza Place  0.05
4                Park  0.05


----Ionview,Kennedy Park,East Birchmount Park----
                venue  freq
0      Discount Store  0.29
1    Department Store  0.14
2  Chinese Restaurant  0.14
3         Coffee Shop  0.14
4          Hobby Shop  0.14


----Kensington Market,Grange Park,Chinatown----
                   venue  freq
0                   Café  0.10
1     Mexican Restaurant  0.07
2  Vietnamese Restaurant  0.07
3   Caribbean Restaurant  0.07
4                 Bakery  0.07


----Kingsview Village,Martin Grove Gardens,Richview Gardens,St. Phillips----
                 venue  freq
0          Pizza Place  0.33
1                 Park  0

                 venue  freq
0                 Pool  0.33
1                 Park  0.33
2                River  0.33
3    Accessories Store  0.00
4  Monument / Landmark  0.00


----Thorncliffe Park----
               venue  freq
0     Sandwich Place  0.12
1  Indian Restaurant  0.12
2                Gym  0.06
3               Bank  0.06
4    Warehouse Store  0.06


----Victoria Hotel,Commerce Court----
           venue  freq
0           Café  0.17
1     Restaurant  0.10
2  Deli / Bodega  0.10
3          Hotel  0.10
4            Gym  0.07


----Victoria Village----
                   venue  freq
0            Coffee Shop   0.2
1           Hockey Arena   0.2
2  Portuguese Restaurant   0.2
3           Intersection   0.2
4            Pizza Place   0.2


----West Deane Park,Cloverdale,Islington,Martin Grove,Princess Gardens----
               venue  freq
0        Golf Course   0.5
1               Bank   0.5
2  Accessories Store   0.0
3              Motel   0.0
4  Martial Arts Dojo   0.0


----W

Let's put that into a pandas dataframe
First, let's write a function to sort the venues in descending order.


In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#Now let's create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']

for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",American Restaurant,Steakhouse,Asian Restaurant,Hotel,Café,Sushi Restaurant,Bar,Speakeasy,Seafood Restaurant,Gastropub
1,Agincourt,Lounge,Clothing Store,Skating Rink,Breakfast Spot,Department Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore
2,"Bathurst Quay,Harbourfront West,Island airport...",Airport Service,Airport Terminal,Airport Lounge,Boutique,Plane,Sculpture Garden,Boat or Ferry,Harbor / Marina,Airport Gate,Airport Food Court
3,Bayview Village,Chinese Restaurant,Bank,Japanese Restaurant,Café,Yoga Studio,Department Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
4,"Bedford Park,Lawrence Manor East",Italian Restaurant,Coffee Shop,Fast Food Restaurant,Sushi Restaurant,Comfort Food Restaurant,Pet Store,Pharmacy,Pizza Place,Café,Pub
5,Berczy Park,Farmers Market,Bakery,Cocktail Bar,Seafood Restaurant,Café,Thai Restaurant,Steakhouse,Bagel Shop,Italian Restaurant,Fish Market
6,Business reply mail Processing Centre969 Eastern,Light Rail Station,Park,Smoke Shop,Spa,Brewery,Farmers Market,Fast Food Restaurant,Burrito Place,Restaurant,Recording Studio
7,"Cabbagetown,St. James Town",Restaurant,Italian Restaurant,Café,Indian Restaurant,Coffee Shop,Pet Store,Bakery,Bank,Butcher,Japanese Restaurant
8,Caledonia-Fairbanks,Park,Pharmacy,Women's Store,Market,Fast Food Restaurant,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore
9,Canada Post Gateway Processing Centre,Coffee Shop,Hotel,Mediterranean Restaurant,Gym / Fitness Center,Middle Eastern Restaurant,Fried Chicken Joint,Burrito Place,American Restaurant,Sandwich Place,Diner


Cluster Neighborhoods
Run k-means to cluster the neighborhood into 5 clusters.


In [31]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 




array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0], dtype=int32)

In [32]:
#Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
toronto_merged = neighborhoods
toronto_merged.rename(columns={'Neighbourhood':'Neighborhood'}, inplace=True)
toronto_merged.shape

(100, 5)

In [33]:
# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353,0,Fast Food Restaurant,Print Shop,Yoga Studio,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,0,Bar,Yoga Studio,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run
2,M1E,Scarborough,"West Hill,Morningside,Guildwood",43.763573,-79.188711,0,Electronics Store,Pizza Place,Breakfast Spot,Medical Center,Mexican Restaurant,Rental Car Location,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Drugstore
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Korean Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Bakery,Bank,Athletics & Sports,Hakka Restaurant,Eastern European Restaurant,Dumpling Restaurant,Drugstore


In [34]:
#Finally, let's visualize the resulting clusters

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


# Examine Clusters

#### Cluster 1

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0,Fast Food Restaurant,Print Shop,Yoga Studio,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store
1,Scarborough,0,Bar,Yoga Studio,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run
2,Scarborough,0,Electronics Store,Pizza Place,Breakfast Spot,Medical Center,Mexican Restaurant,Rental Car Location,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Drugstore
3,Scarborough,0,Coffee Shop,Korean Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run
4,Scarborough,0,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Bakery,Bank,Athletics & Sports,Hakka Restaurant,Eastern European Restaurant,Dumpling Restaurant,Drugstore
5,Scarborough,0,Playground,Yoga Studio,Dance Studio,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store
6,Scarborough,0,Discount Store,Department Store,Train Station,Coffee Shop,Chinese Restaurant,Hobby Shop,Dessert Shop,Dim Sum Restaurant,Diner,Yoga Studio
7,Scarborough,0,Bus Line,Bakery,Soccer Field,Ice Cream Shop,Intersection,Metro Station,Bus Station,Park,Drugstore,Dumpling Restaurant
9,Scarborough,0,College Stadium,General Entertainment,Skating Rink,Café,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore
10,Scarborough,0,Indian Restaurant,Vietnamese Restaurant,Gaming Cafe,Chinese Restaurant,Latin American Restaurant,Pet Store,Brewery,Eastern European Restaurant,Dumpling Restaurant,Drugstore


#### Cluster 2

In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Scarborough,1,Motel,American Restaurant,Skating Rink,Movie Theater,Yoga Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
24,North York,1,Park,Pizza Place,Coffee Shop,Pharmacy,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run
28,North York,1,Coffee Shop,Sushi Restaurant,Pharmacy,Pizza Place,Middle Eastern Restaurant,Deli / Bodega,Restaurant,Bridal Shop,Sandwich Place,Diner
31,North York,1,Grocery Store,Bank,Shopping Mall,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run
49,Central Toronto,1,Coffee Shop,Pub,American Restaurant,Fried Chicken Joint,Light Rail Station,Supermarket,Vietnamese Restaurant,Pizza Place,Bagel Shop,Sushi Restaurant
50,Downtown Toronto,1,Park,Playground,Trail,Yoga Studio,Diner,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Dog Run
52,Downtown Toronto,1,Burger Joint,Gastropub,Ethiopian Restaurant,Hobby Shop,Dance Studio,Pub,Ramen Restaurant,Ice Cream Shop,Indian Restaurant,Japanese Restaurant
57,Downtown Toronto,1,Coffee Shop,Spa,Italian Restaurant,Café,Bubble Tea Shop,Ice Cream Shop,Bar,Seafood Restaurant,Sandwich Place,Ramen Restaurant
60,Downtown Toronto,1,Coffee Shop,Café,Deli / Bodega,Restaurant,Art Gallery,Pizza Place,Steakhouse,Bakery,Pub,Hotel Bar
69,Downtown Toronto,1,Café,Cocktail Bar,Seafood Restaurant,Farmers Market,Clothing Store,Park,Pub,Steakhouse,Jazz Club,Beer Bar


#### Cluster 3

In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,2,Park,Dim Sum Restaurant,Swim School,Bus Line,Yoga Studio,Deli / Bodega,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
84,West Toronto,2,Coffee Shop,Café,Italian Restaurant,Sushi Restaurant,Gourmet Shop,Food,Scenic Lookout,Latin American Restaurant,Bookstore,Diner


#### Cluster 4

In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
102,Etobicoke,3,Drugstore,Rental Car Location,Yoga Studio,Dance Studio,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store


#### Cluster 5

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
78,West Toronto,4,Coffee Shop,Breakfast Spot,Café,Pet Store,Stadium,Burrito Place,Caribbean Restaurant,Climbing Gym,Performing Arts Venue,Bakery
